### imports

In [1]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.insert(0, os.path.abspath(''))
from python_scripts.utilities.api_calls import get_cookie_from_blob, fetch_items, fetch_item_to_df


### setting portfolio information

In [8]:
def initial_portfolio():
    initial_balance = 10000  
    # item selection (chose your own in custom_items or leave def blank for default items)
    custom_items = ["AK-47 | Redline (Field-Tested)", "AWP | Asiimov (Field-Tested)", "M4A4 | Neo-Noir (Factory New)"]
    selected_items = fetch_items(custom_items)
    
    # Set allocation. (equal weight)
    allocations = {item: 1/len(selected_items) for item in selected_items}
    
    # Dictionary for hold portfolio info
    portfolio_info = {
        'initial_balance': initial_balance,
        'current_balance': initial_balance,
        'items': selected_items,
        'allocations': allocations,
        'holdings': {item: 0 for item in selected_items},
    }
    
    # Load data for items in portfolio 
    item_data = {}
    dailyCookie = get_cookie_from_blob()
    for item in portfolio_info['items']:
        item_data[item] = fetch_item_to_df(item, dailyCookie)
        print(f"Data loaded for {item}")

    return portfolio_info, item_data

portfolio_info, item_data = initial_portfolio()
# print("Portfolio Info:", portfolio_info)

# Item Data info
for item, data in item_data.items():
    print(f"\n{item}:")
    print(f"Data shape: {data.shape}")
    print(f"Date range: {data.index.min()} to {data.index.max()}")
    print(data.head())


Data loaded for AK-47 | Redline (Field-Tested)
Data loaded for AWP | Asiimov (Field-Tested)
Data loaded for M4A4 | Neo-Noir (Factory New)

AK-47 | Redline (Field-Tested):
Data shape: (3851, 2)
Date range: 2014-02-21 00:00:00 to 2024-09-06 00:00:00
            price_usd  volume
date                         
2014-02-21     31.477     198
2014-02-22     26.668     225
2014-02-23     25.200     183
2014-02-24     27.570     128
2014-02-25     31.120      90

AWP | Asiimov (Field-Tested):
Data shape: (3851, 2)
Date range: 2014-02-21 00:00:00 to 2024-09-06 00:00:00
            price_usd  volume
date                         
2014-02-21    125.620      16
2014-02-22     79.906      38
2014-02-23     87.730      16
2014-02-24     97.697      12
2014-02-25    110.601      13

M4A4 | Neo-Noir (Factory New):
Data shape: (2395, 2)
Date range: 2018-02-16 00:00:00 to 2024-09-06 00:00:00
            price_usd  volume
date                         
2018-02-16    286.243       8
2018-02-17    241.857    

### running it back

In [12]:
# Usage
start_date = '2023-01-01'  # Define a start date
end_date = '2024-01-01'  # Example end date
performance = simulate_portfolio(portfolio_info, item_data, start_date, end_date)


def buy_items(portfolio_info, item_data, date):
    remaining_balance = portfolio_info['initial_balance']

    for item, allocation in portfolio_info['allocations'].items():
        amount_to_spend = portfolio_info['initial_balance'] * allocation
        price = item_data[item].loc[date, 'price_usd']
        quantity = amount_to_spend // price  # Integer division for whole number of items
        cost = quantity * price
        remaining_balance -= cost
        portfolio_info['holdings'][item] = quantity
    portfolio_info['current_balance'] = remaining_balance

def sell_items(portfolio_info, item_data, date):
    total_value = portfolio_info['current_balance']
    for item, quantity in portfolio_info['holdings'].items():
        price = item_data[item].loc[date, 'price_usd']
        value = quantity * price
        total_value += value
    return total_value

def simulate_portfolio(portfolio_info, item_data, start_date, end_date):
    # Set start and end dates
    portfolio_info['start_date'] = pd.to_datetime(start_date)
    portfolio_info['present_date'] = pd.to_datetime(end_date)
    
    # BUY items at start date
    buy_items(portfolio_info, item_data, portfolio_info['start_date'])
    
    # SELL items at end date
    final_value = sell_items(portfolio_info, item_data, portfolio_info['present_date'])
    
    return calculate_performance(portfolio_info, final_value)

def calculate_performance(portfolio_info, final_value):
    initial_balance = portfolio_info['initial_balance']
    total_return = (final_value - initial_balance) / initial_balance
    days_held = (portfolio_info['present_date'] - portfolio_info['start_date']).days
    annualized_return = (1 + total_return) ** (365 / days_held) - 1

    return {
        'initial_balance': initial_balance,
        'final_value': final_value,
        'total_return': total_return,
        'annualized_return': annualized_return,
        'days_held': days_held
    }


print(f"Initial Balance: ${performance['initial_balance']:.2f}")
print(f"Final Value: ${performance['final_value']:.2f}")
print(f"Total Return: {performance['total_return']:.2%}")
print(f"Annualized Return: {performance['annualized_return']:.2%}")
print(f"Days Held: {performance['days_held']}")

Initial Balance: $10000.00
Final Value: $10769.65
Total Return: 7.70%
Annualized Return: 7.70%
Days Held: 365
